In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
import anndata

adata = anndata.read_h5ad("../../data/raw/bowel_disease.h5ad")

In [4]:
from mmcontext.models.geneformer_model import GeneformerTranscriptomeProcessor

processor = GeneformerTranscriptomeProcessor(emb_label={"cluster_label": "cluster_label"}, nproc=16)

In [7]:
dataset = processor(adata)

In [6]:
# from mmcontext.pp.GeneformerProcessor import GeneformerProcessor
# model_path = "../../../Geneformer/gf-6L-30M-i2048"
# processor = GeneformerProcessor(model_path, token_dict_path = "../../../Geneformer/gene_dictionaries_30m/ensembl_mapping_dict_gc30M.pkl")

In [9]:
from datasets import Dataset

hf_dataset = Dataset.from_dict(dataset)

In [9]:
import torch
from geneformer.perturber_utils import pad_tensor_list

PAD_TOKEN_ID = 0
MODEL_INPUT_SIZE = 4096
max_len = max(hf_dataset["expression_token_lengths"])
tokens = [torch.tensor(v).to(dtype=torch.long) for v in hf_dataset["expression_tokens"]]
tokens = pad_tensor_list(
    tokens,
    dynamic_or_constant=None,  # need to set to None to actually use model_input_size
    pad_token_id=PAD_TOKEN_ID,
    model_input_size=MODEL_INPUT_SIZE,
)
expression_token_lengths = torch.tensor(hf_dataset["expression_token_lengths"])

If padding style is constant, must provide integer value. Setting padding to max input size 4096.


In [10]:
input_tensor = tokens

In [11]:
from tdc import tdc_hf_interface

# retrieve model
geneformer = tdc_hf_interface("Geneformer")
model = geneformer.load()

In [13]:
input_tensor.shape[0]

11175

In [12]:
processor.tokenizer.gene_token_dict

{'<pad>': 0,
 '<mask>': 1,
 '<cls>': 2,
 '<eos>': 3,
 'ENSG00000000003': 4,
 'ENSG00000000005': 5,
 'ENSG00000000419': 6,
 'ENSG00000000457': 7,
 'ENSG00000000460': 8,
 'ENSG00000000938': 9,
 'ENSG00000000971': 10,
 'ENSG00000001036': 11,
 'ENSG00000001084': 12,
 'ENSG00000001167': 13,
 'ENSG00000001460': 14,
 'ENSG00000001461': 15,
 'ENSG00000001497': 16,
 'ENSG00000001561': 17,
 'ENSG00000001617': 18,
 'ENSG00000001626': 19,
 'ENSG00000001629': 20,
 'ENSG00000001630': 21,
 'ENSG00000001631': 22,
 'ENSG00000002016': 23,
 'ENSG00000002330': 24,
 'ENSG00000002549': 25,
 'ENSG00000002586': 26,
 'ENSG00000002587': 27,
 'ENSG00000002726': 28,
 'ENSG00000002745': 29,
 'ENSG00000002746': 30,
 'ENSG00000002822': 31,
 'ENSG00000002834': 32,
 'ENSG00000002919': 33,
 'ENSG00000002933': 34,
 'ENSG00000003056': 35,
 'ENSG00000003096': 36,
 'ENSG00000003137': 37,
 'ENSG00000003147': 38,
 'ENSG00000003249': 39,
 'ENSG00000003393': 40,
 'ENSG00000003400': 41,
 'ENSG00000003402': 42,
 'ENSG00000003436

In [18]:
from geneformer.perturber_utils import quant_layers

for batch in range(0, input_tensor.shape[0], 16):
    input = input_tensor[batch : batch + 16]
    attention_mask = torch.tensor(
        [[x[0] != 0, x[1] != 0] for x in input]
    )  # here we assume we used 0/False as a special padding token
    outputs = model(input, attention_mask=attention_mask, output_hidden_states=True)
    layer_to_quant = quant_layers(model) + (-1)  # Geneformer's second-to-last layer is most generalized
    embs_i = outputs.hidden_states[layer_to_quant]
    # there are "cls", "cell", and "gene" embeddings. we will only capture "gene", which is cell type specific. for "cell", you'd average out across unmasked gene embeddings per cell
    embs = embs_i
    break